In [1]:
import os

In [2]:
%pwd

'd:\\software\\python_vs\\lone_approval_prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\software\\python_vs\\lone_approval_prediction'

## Entity

In [5]:
from dataclasses import dataclass
from pathlib  import Path


@dataclass
class DataValidationconfig:
    root_dir: Path
    unzip_dir: Path
    STATUS_FILE: str
    all_schema: dict
    
    

## configuration manager

In [6]:
from lone_approval_prediction.constants import *
from lone_approval_prediction.utils.comman import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
    
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
    
        create_directories([self.config.artifacts_root])
        
    def get_data_validation_config(self) -> DataValidationconfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        
        create_directories([config.root_dir])
        
        data_validation_config = DataValidationconfig(
            root_dir= config.root_dir,
            unzip_dir= config.unzip_dir,
            STATUS_FILE= config.STATUS_FILE,
            all_schema= schema
    
        )
        
        return data_validation_config
    
    

## Components

In [14]:
import os
import pandas as pd
import numpy as np
from lone_approval_prediction import logger

class DataValidation:
    def __init__(self,config):
        self.config = config
    
    def validate_all_columns(self) -> bool:
        try:
            validation_status = None
            
            data = pd.read_csv(self.config.unzip_dir)
            all_colls = list(data.columns)
            
            all_schema = list(self.config.all_schema.keys())
            
            for col in all_colls:
                if col.replace(" ","") not in all_schema and col.replace(" ","") != "loan_status":
                    validation_status = False
                    with open(self.config.STATUS_FILE, "w") as file:
                        file.write(f" validation status : {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, "w") as file:
                        file.write(f" validation status : {validation_status}")
                        
            return validation_status
        
        except Exception as e:
            logger.error(f" error during validation {e}")
            raise e
                    
                

## Pipeline

In [15]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

2025-02-04 21:17:21,282: INFO: comman: yaml file : config\config.yaml lodded successfully
2025-02-04 21:17:21,287: INFO: comman: yaml file : params.yaml lodded successfully
2025-02-04 21:17:21,290: INFO: comman: yaml file : schema.yaml lodded successfully
2025-02-04 21:17:21,293: INFO: comman: created directory at :artifacts
2025-02-04 21:17:21,295: INFO: comman: created directory at :artifacts/data_validation
